In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve

In [2]:
def score(y,pred): 
    fpr, tpr, thresholds = roc_curve(y, pred, pos_label=1) 
    score=0.4*tpr[np.where(fpr>=0.001)[0][0]]+0.3*tpr[np.where(fpr>=0.005)[0][0]]+0.3*tpr[np.where(fpr>=0.01)[0][0]] 
    return score 

In [3]:
DATA_PATH = "datasets"
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_train.csv")
data = pd.read_csv(csv_path)

In [4]:
pd.set_option('display.max_columns',2000)
data

,id,label,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,f10eb20f31cf7063ee8bdbd1272214e4d7e0193c8dbce4...,0,20171103,0,0,0,0,100807.0,0,5,1,1,1,1,2,0,2,0,2,2,2,1,31.0,61.0,142.0,245.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,3.0,96.0,38.0,301.0,302.0,302.0,302.0,301.0,302.0,40.0,79.0,40.0,79.0,2.0,3.0,4.0,3.0,41.0,80.0,96.5,721.1,3252.0,154.5,394.5,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,4.0,8.0,15.0,30.0,4.0,14.0,34.0,70.0,142.0,3.0,15.0,41.0,79.0,191.0,192.0,190.0,190.0,190.0,190.0,191.0,192.0,191.0,190.0,190.0,190.0,32.0,33.0,25.0,25.0,24.0,25.0,31.0,79.0,42.0,42.0,42.0,68.0,32.0,32.0,28.0,27.0,28.0,27.0,32.0,235.0,235.0,126.0,126.0,127.0,126.0,235.0,192.0,191.0,190.0,189.0,189.0,301.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,3.0,4.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,4.0,3.0,4.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,3.0,3.0,3.0,14.0,41.0,80.0,167.0,2.0,2.0,4.0,3.0,3.0,4.0,6.0,6.0,2.0,2.0,4.0,3.0,2.0,2.0,3.0,4.0,3.0,3.0,4.0,1.0,3.0,3.0,4.0,2.0,2.0,2.0,2.0,8.0,2.0,2.0,3.0,3.0,3.0,2.0,5.0,5.0,10.0,16.0,2.0,2.0,4.0,5.0,7.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,27.0,297.0,302.0,384.0,770.0,20.0,99.0,102.0,124.0,140.0,301.0,312.0,328.0,85.0,302.0,201.0,203.0,203.0,61.0,201.0
1,d861929b67938d06538b910b9f6b85f5eb62b6ad7361ba...,0,20170917,0,1,1,1,100805.0,1,5,2,1,1,0,2,1,2,2,1,1,2,0,14.0,13.0,48.0,48.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,1.0,1.0,2.0,4.0,4.0,5.0,1.0,1.0,4.0,3.0,3.0,79.0,12.0,302.0,32.0,301.0,302.0,302.0,302.0,23.0,26.0,23.0,26.0,2.0,2.0,3.0,4.0,23.0,26.0,5.0,717.4,733.3,231.8,472.8,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,4.0,8.0,11.0,12.0,4.0,12.0,23.0,26.0,27.0,5.0,12.0,24.0,27.0,122.0,121.0,16.0,121.0,121.0,122.0,121.0,122.0,121.0,16.0,121.0,122.0,32.0,31.0,13.0,27.0,26.0,27.0,32.0,79.0,40.0,39.0,39.0,63.0,31.0,31.0,15.0,28.0,29.0,29.0,31.0,231.0,232.0,17.0,99.0,99.0,98.0,232.0,121.0,122.0,17.0,121.0,121.0,302.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,4.0,5.0,12.0,24.0,26.0,27.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1

In [5]:
#############数据预处理#######################
data.loc[data['label'] == -1, 'label'] = 1
data.fillna(0, inplace=True)
# data.drop('f6',axis=1, inplace=True)
# data.drop('f58',axis=1, inplace=True)
# data.drop('f112',axis=1, inplace=True)
# data.drop('f113',axis=1, inplace=True)
# data.drop('f114',axis=1, inplace=True)
# data.drop('f115',axis=1, inplace=True)
# data.drop('f116',axis=1, inplace=True)
# data.drop('f117',axis=1, inplace=True)
# data.drop('f118',axis=1, inplace=True)
# data.drop('f119',axis=1, inplace=True)
# data.drop('f121',axis=1, inplace=True)
# data.drop('f122',axis=1, inplace=True)
# data.drop('f128',axis=1, inplace=True)
# data.drop('f132',axis=1, inplace=True)
# data.drop('f133',axis=1, inplace=True)
# data.drop('f145',axis=1, inplace=True)
# data.drop('f146',axis=1, inplace=True)
# data.drop('f147',axis=1, inplace=True)
# data.drop('f153',axis=1, inplace=True)
# data.drop('f221',axis=1, inplace=True)
# data.drop('f294',axis=1, inplace=True)
# means = []
# cols = [col for col in data.loc[:, 'f1':].columns]
# for col in cols:
#     mean = data[col].mean()
#     data[col].fillna(mean, inplace=True)   
#     means.append(mean)

In [6]:
data_test = data[data['date'] > 20171031]
data_train = data[data['date'] <= 20171025]
data_val = data[(data['date'] <= 20171031) & (data['date'] > 20171025)]
print(data.shape)
print(data_train.shape)
print(data_test.shape)
print(data_val.shape)

(994731, 300)
(814042, 300)
(83125, 300)
(97564, 300)


In [7]:
X = data.loc[:,'date':].as_matrix()
y = data.loc[:,'label'].as_matrix()
X_train = data_train.loc[:,'date':].as_matrix()
y_train = data_train.loc[:,'label'].as_matrix()
X_val = data_val.loc[:,'date':].as_matrix()
y_val = data_val.loc[:,'label'].as_matrix()
X_test = data_test.loc[:,'date':].as_matrix()
y_test = data_test.loc[:,'label'].as_matrix()
print(X.shape)
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

(994731, 298)
(814042, 298)
(814042,)
(97564, 298)
(97564,)
(83125, 298)
(83125,)


In [103]:
##########线下测试############
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [104]:
##########线下测试############
#X_train_1, X_val, y_train_1, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=0)

In [38]:
##########线下测试：调参############
# print('数据转换')
# lgb_train = lgb.Dataset(X_train, y_train, free_raw_data=False)   #free_raw_data (bool, optional (default=True)) 
#                                                                 #If True, raw data is freed after constructing inner Dataset.
# lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train, free_raw_data=False)

# print('设置参数')
# params = {
#           'boosting_type': 'gbdt',
#           'objective': 'binary',
#           'metric': {'binary_logloss', 'auc'},
#           }

# ### 交叉验证(调参)
# print('交叉验证')
# min_merror = float('Inf')
# best_params = {}

# print("调参1：提高准确率")
# for num_leaves in range(20,200,5):
#     for max_depth in range(3,8,1):
#         params['num_leaves'] = num_leaves
#         params['max_depth'] = max_depth

#         cv_results = lgb.cv(
#                             params,
#                             lgb_train,
#                             seed=2018,
#                             nfold=3,
#                             metrics=['binary_error'],
#                             early_stopping_rounds=10,
#                             verbose_eval=True
#                             )

#         mean_merror = pd.Series(cv_results['binary_error-mean']).min()
#         boost_rounds = pd.Series(cv_results['binary_error-mean']).argmin()

#         if mean_merror < min_merror:
#             min_merror = mean_merror
#             best_params['num_leaves'] = num_leaves
#             best_params['max_depth'] = max_depth

# params['num_leaves'] = best_params['num_leaves']
# params['max_depth'] = best_params['max_depth']

数据转换
设置参数
交叉验证
调参1：提高准确率
[1]	cv_agg's binary_error: 0.0158726 + 0.000119445
[2]	cv_agg's binary_error: 0.0157301 + 5.4585e-05
[3]	cv_agg's binary_error: 0.0156736 + 0.000162556
[4]	cv_agg's binary_error: 0.0157854 + 5.56746e-05
[5]	cv_agg's binary_error: 0.0157375 + 4.08263e-05
[6]	cv_agg's binary_error: 0.0155999 + 6.59732e-05
[7]	cv_agg's binary_error: 0.01567 + 5.35085e-05
[8]	cv_agg's binary_error: 0.0156614 + 3.07624e-05
[9]	cv_agg's binary_error: 0.0156331 + 1.37944e-05
[10]	cv_agg's binary_error: 0.015627 + 6.3882e-05
[11]	cv_agg's binary_error: 0.0156122 + 3.59154e-05
[12]	cv_agg's binary_error: 0.015579 + 1.93358e-05
[13]	cv_agg's binary_error: 0.0155127 + 3.81278e-05
[14]	cv_agg's binary_error: 0.0155201 + 1.65994e-05
[15]	cv_agg's binary_error: 0.0154992 + 3.21893e-05
[16]	cv_agg's binary_error: 0.0154501 + 6.69149e-05
[17]	cv_agg's binary_error: 0.0154439 + 6.99542e-05
[18]	cv_agg's binary_error: 0.0154009 + 7.73804e-05
[19]	cv_agg's binary_error: 0.0153604 + 0.00010192
[20

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: FutureWarning: 'argmin' is deprecated. Use 'idxmin' instead. The behavior of 'argmin' will be corrected to return the positional minimum in the future. Use 'series.values.argmin' to get the position of the minimum now.


[1]	cv_agg's binary_error: 0.0152474 + 7.33747e-05
[2]	cv_agg's binary_error: 0.0153727 + 7.76471e-05
[3]	cv_agg's binary_error: 0.01524 + 6.63836e-05
[4]	cv_agg's binary_error: 0.0152707 + 8.87482e-05
[5]	cv_agg's binary_error: 0.0151958 + 4.40565e-05
[6]	cv_agg's binary_error: 0.0151491 + 3.31712e-05
[7]	cv_agg's binary_error: 0.0151098 + 2.87259e-05
[8]	cv_agg's binary_error: 0.0150729 + 5.02854e-05
[9]	cv_agg's binary_error: 0.0150594 + 5.28607e-05
[10]	cv_agg's binary_error: 0.0150533 + 9.82857e-05
[11]	cv_agg's binary_error: 0.0150791 + 5.1474e-05
[12]	cv_agg's binary_error: 0.015068 + 2.21975e-05
[13]	cv_agg's binary_error: 0.0150582 + 4.69325e-05
[14]	cv_agg's binary_error: 0.0150643 + 4.06543e-05
[15]	cv_agg's binary_error: 0.0150226 + 3.64174e-05
[16]	cv_agg's binary_error: 0.0150287 + 2.57305e-05
[17]	cv_agg's binary_error: 0.015025 + 2.08734e-05
[18]	cv_agg's binary_error: 0.0150103 + 3.17258e-05
[19]	cv_agg's binary_error: 0.014971 + 5.0041e-05
[20]	cv_agg's binary_error: 

[60]	cv_agg's binary_error: 0.0138285 + 5.04946e-05
[61]	cv_agg's binary_error: 0.0137966 + 3.71616e-05
[62]	cv_agg's binary_error: 0.0137585 + 5.12043e-05
[63]	cv_agg's binary_error: 0.0137155 + 4.68614e-05
[64]	cv_agg's binary_error: 0.0136909 + 6.64234e-05
[65]	cv_agg's binary_error: 0.0136369 + 4.8197e-05
[66]	cv_agg's binary_error: 0.0136283 + 5.92167e-05
[67]	cv_agg's binary_error: 0.0135742 + 5.28584e-05
[68]	cv_agg's binary_error: 0.0135276 + 3.88246e-05
[69]	cv_agg's binary_error: 0.0134919 + 3.65012e-05
[70]	cv_agg's binary_error: 0.0134907 + 4.55826e-05
[71]	cv_agg's binary_error: 0.0134772 + 1.59301e-05
[72]	cv_agg's binary_error: 0.0134723 + 3.21704e-05
[73]	cv_agg's binary_error: 0.0134244 + 4.57879e-06
[74]	cv_agg's binary_error: 0.0134268 + 1.76464e-05
[75]	cv_agg's binary_error: 0.0134133 + 9.69574e-06
[76]	cv_agg's binary_error: 0.0134145 + 1.08661e-05
[77]	cv_agg's binary_error: 0.0133875 + 2.73116e-05
[78]	cv_agg's binary_error: 0.0133789 + 2.00483e-05
[79]	cv_agg's

[19]	cv_agg's binary_error: 0.0146651 + 0.000117979
[20]	cv_agg's binary_error: 0.0146491 + 0.000127266
[21]	cv_agg's binary_error: 0.0145766 + 0.000126754
[22]	cv_agg's binary_error: 0.0145472 + 8.41719e-05
[23]	cv_agg's binary_error: 0.0144501 + 0.000131054
[24]	cv_agg's binary_error: 0.0144219 + 0.00013124
[25]	cv_agg's binary_error: 0.014412 + 0.00011939
[26]	cv_agg's binary_error: 0.0143887 + 0.000140243
[27]	cv_agg's binary_error: 0.0143076 + 8.45787e-05
[28]	cv_agg's binary_error: 0.0142904 + 5.67825e-05
[29]	cv_agg's binary_error: 0.0142585 + 7.35069e-05
[30]	cv_agg's binary_error: 0.0141946 + 3.47578e-05
[31]	cv_agg's binary_error: 0.0141565 + 5.42053e-05
[32]	cv_agg's binary_error: 0.014143 + 2.84239e-05
[33]	cv_agg's binary_error: 0.0140902 + 4.22475e-05
[34]	cv_agg's binary_error: 0.0140668 + 5.85636e-05
[35]	cv_agg's binary_error: 0.0139919 + 3.07204e-05
[36]	cv_agg's binary_error: 0.0139637 + 3.97901e-05
[37]	cv_agg's binary_error: 0.0139403 + 2.77843e-05
[38]	cv_agg's bi

...

[71]	cv_agg's binary_error: 0.0127831 + 3.01207e-05
[72]	cv_agg's binary_error: 0.0127733 + 1.9714e-05
[73]	cv_agg's binary_error: 0.0127647 + 2.71172e-05
[74]	cv_agg's binary_error: 0.0127819 + 7.49226e-05
[75]	cv_agg's binary_error: 0.0127635 + 9.11144e-05
[76]	cv_agg's binary_error: 0.0127647 + 8.45707e-05
[77]	cv_agg's binary_error: 0.0127684 + 8.16296e-05
[78]	cv_agg's binary_error: 0.0127647 + 8.94599e-05
[79]	cv_agg's binary_error: 0.0127414 + 0.000111326
[80]	cv_agg's binary_error: 0.0127303 + 8.80321e-05
[81]	cv_agg's binary_error: 0.0127168 + 8.86984e-05
[82]	cv_agg's binary_error: 0.0126984 + 9.52433e-05
[83]	cv_agg's binary_error: 0.0126922 + 8.09438e-05
[84]	cv_agg's binary_error: 0.012675 + 5.58963e-05
[85]	cv_agg's binary_error: 0.0126443 + 2.70624e-05
[86]	cv_agg's binary_error: 0.0126296 + 2.56915e-05
[87]	cv_agg's binary_error: 0.0126296 + 2.40534e-05
[88]	cv_agg's binary_error: 0.0126185 + 2.60482e-05
[89]	cv_agg's binary_error: 0.0126345 + 3.0293e-05
[90]	cv_agg's b

[30]	cv_agg's binary_error: 0.0153358 + 3.67662e-05
[31]	cv_agg's binary_error: 0.0153039 + 6.90442e-05
[32]	cv_agg's binary_error: 0.0152658 + 6.02987e-05
[33]	cv_agg's binary_error: 0.0153002 + 5.05804e-05
[34]	cv_agg's binary_error: 0.0152867 + 8.76961e-05
[35]	cv_agg's binary_error: 0.0152646 + 9.68734e-05
[36]	cv_agg's binary_error: 0.0152756 + 6.65577e-05
[37]	cv_agg's binary_error: 0.0152388 + 6.55964e-05
[38]	cv_agg's binary_error: 0.0152302 + 5.0755e-05
[39]	cv_agg's binary_error: 0.0152105 + 4.81994e-05
[40]	cv_agg's binary_error: 0.0151909 + 6.22031e-05
[41]	cv_agg's binary_error: 0.0151577 + 7.68007e-05
[42]	cv_agg's binary_error: 0.0151454 + 7.56921e-05
[43]	cv_agg's binary_error: 0.0151344 + 8.34866e-05
[44]	cv_agg's binary_error: 0.0151147 + 8.17334e-05
[45]	cv_agg's binary_error: 0.0151147 + 8.42973e-05
[46]	cv_agg's binary_error: 0.0151196 + 5.38817e-05
[47]	cv_agg's binary_error: 0.0150889 + 6.14969e-05
[48]	cv_agg's binary_error: 0.0150729 + 4.21525e-05
[49]	cv_agg's

[89]	cv_agg's binary_error: 0.0137425 + 4.13901e-05
[90]	cv_agg's binary_error: 0.0137462 + 2.71021e-05
[91]	cv_agg's binary_error: 0.0137376 + 3.67892e-05
[92]	cv_agg's binary_error: 0.0137032 + 7.57538e-06
[93]	cv_agg's binary_error: 0.013702 + 2.86937e-05
[94]	cv_agg's binary_error: 0.0136799 + 2.10722e-05
[95]	cv_agg's binary_error: 0.0136737 + 1.54466e-05
[96]	cv_agg's binary_error: 0.013659 + 1.65626e-05
[97]	cv_agg's binary_error: 0.013643 + 1.50247e-05
[98]	cv_agg's binary_error: 0.0136197 + 2.00126e-05
[99]	cv_agg's binary_error: 0.0136135 + 1.59069e-05
[100]	cv_agg's binary_error: 0.0136049 + 6.93731e-06
[1]	cv_agg's binary_error: 0.0150643 + 4.72525e-05
[2]	cv_agg's binary_error: 0.0150434 + 4.4346e-05
[3]	cv_agg's binary_error: 0.0149525 + 2.11583e-05
[4]	cv_agg's binary_error: 0.0149169 + 2.45174e-05
[5]	cv_agg's binary_error: 0.0148297 + 1.59477e-05
[6]	cv_agg's binary_error: 0.0148395 + 4.83799e-05
[7]	cv_agg's binary_error: 0.0148039 + 3.47252e-05
[8]	cv_agg's binary_er

[48]	cv_agg's binary_error: 0.0133286 + 3.80786e-05
[49]	cv_agg's binary_error: 0.0132684 + 3.80862e-05
[50]	cv_agg's binary_error: 0.0132352 + 4.00485e-05
[51]	cv_agg's binary_error: 0.0132192 + 4.63017e-05
[52]	cv_agg's binary_error: 0.0131762 + 4.35131e-05
[53]	cv_agg's binary_error: 0.0131431 + 5.93503e-05
[54]	cv_agg's binary_error: 0.0131259 + 3.8079e-05
[55]	cv_agg's binary_error: 0.0130608 + 4.77448e-05
[56]	cv_agg's binary_error: 0.0130337 + 8.01749e-05
[57]	cv_agg's binary_error: 0.0130202 + 7.09306e-05
[58]	cv_agg's binary_error: 0.0130042 + 6.10791e-05
[59]	cv_agg's binary_error: 0.012933 + 3.67542e-05
[60]	cv_agg's binary_error: 0.0128925 + 1.42052e-05
[61]	cv_agg's binary_error: 0.01289 + 4.08418e-05
[62]	cv_agg's binary_error: 0.0128937 + 2.21871e-05
[63]	cv_agg's binary_error: 0.0128753 + 3.61514e-05
[64]	cv_agg's binary_error: 0.0128703 + 4.38406e-05
[65]	cv_agg's binary_error: 0.0128372 + 3.38045e-05
[66]	cv_agg's binary_error: 0.0128175 + 2.3869e-05
[67]	cv_agg's bin

KeyboardInterrupt: 

In [ ]:
##########线下测试：调参############
# 过拟合
# print("调参2：降低过拟合")
# for max_bin in range(1,255,5):
#     for min_data_in_leaf in range(10,200,5):
#             params['max_bin'] = max_bin
#             params['min_data_in_leaf'] = min_data_in_leaf

#             cv_results = lgb.cv(
#                                 params,
#                                 lgb_train,
#                                 seed=42,
#                                 nfold=3,
#                                 metrics=['binary_error'],
#                                 early_stopping_rounds=3,
#                                 verbose_eval=True
#                                 )

#             mean_merror = pd.Series(cv_results['binary_error-mean']).min()
#             boost_rounds = pd.Series(cv_results['binary_error-mean']).argmin()

#             if mean_merror < min_merror:
#                 min_merror = mean_merror
#                 best_params['max_bin']= max_bin
#                 best_params['min_data_in_leaf'] = min_data_in_leaf

# params['min_data_in_leaf'] = best_params['min_data_in_leaf']
# params['max_bin'] = best_params['max_bin']

In [ ]:
##########线下测试：调参############
# print("调参3：降低过拟合")
# for feature_fraction in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
#     for bagging_fraction in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
#         for bagging_freq in range(0,50,5):
#             params['feature_fraction'] = feature_fraction
#             params['bagging_fraction'] = bagging_fraction
#             params['bagging_freq'] = bagging_freq

#             cv_results = lgb.cv(
#                                 params,
#                                 lgb_train,
#                                 seed=42,
#                                 nfold=3,
#                                 metrics=['binary_error'],
#                                 early_stopping_rounds=3,
#                                 verbose_eval=True
#                                 )

#             mean_merror = pd.Series(cv_results['binary_error-mean']).min()
#             boost_rounds = pd.Series(cv_results['binary_error-mean']).argmin()

#             if mean_merror < min_merror:
#                 min_merror = mean_merror
#                 best_params['feature_fraction'] = feature_fraction
#                 best_params['bagging_fraction'] = bagging_fraction
#                 best_params['bagging_freq'] = bagging_freq

# params['feature_fraction'] = best_params['feature_fraction']
# params['bagging_fraction'] = best_params['bagging_fraction']
# params['bagging_freq'] = best_params['bagging_freq']

In [ ]:
##########线下测试：调参############
# print("调参4：降低过拟合")
# for lambda_l1 in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
#     for lambda_l2 in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
#         for min_split_gain in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
#             params['lambda_l1'] = lambda_l1
#             params['lambda_l2'] = lambda_l2
#             params['min_split_gain'] = min_split_gain

#             cv_results = lgb.cv(
#                                 params,
#                                 lgb_train,
#                                 seed=42,
#                                 nfold=3,
#                                 metrics=['binary_error'],
#                                 early_stopping_rounds=3,
#                                 verbose_eval=True
#                                 )

#             mean_merror = pd.Series(cv_results['binary_error-mean']).min()
#             boost_rounds = pd.Series(cv_results['binary_error-mean']).argmin()

#             if mean_merror < min_merror:
#                 min_merror = mean_merror
#                 best_params['lambda_l1'] = lambda_l1
#                 best_params['lambda_l2'] = lambda_l2
#                 best_params['min_split_gain'] = min_split_gain

# params['lambda_l1'] = best_params['lambda_l1']
# params['lambda_l2'] = best_params['lambda_l2']
# params['min_split_gain'] = best_params['min_split_gain']

In [ ]:
##########线下测试：调参############
print(best_params)

In [7]:
########## 线下测试  RandomForestClassifier ############
data_test = data[data['date'] > 20171031]
data_train = data[data['date'] <= 20171031]
X_train = data_train.loc[:,'date':].as_matrix()
y_train = data_train.loc[:,'label'].as_matrix()
X_test = data_test.loc[:,'date':].as_matrix()
y_test = data_test.loc[:,'label'].as_matrix()
print(X_train.shape)
print(X_test.shape)

(911606, 298)
(83125, 298)


In [10]:
print(score(y_test, y_pred[:, 1]))

0.42195306585919756


In [8]:
########## 线下测试  RandomForestClassifier ############
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
y_pred = rfc.predict_proba(X_test)
print(score(y_test, y_pred[:, 1]))

0.42195306585919756


In [9]:
y_pred[:20]

array([[1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ],
       [0.4, 0.6],
       [1. , 0. ],
       [1. , 0. ],
       [1. , 0. ]])

In [27]:
########## 线下测试  lightGBM ############
# params = {
#     'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': {'l2', 'auc'},
#     'num_leaves': 30,            #
#     'learning_rate': 0.05,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'bagging_freq': 5,
#     'verbose': 0
# }

# params = {
#     'boosting_type': 'rf',
#     # default=gbdt   options=gbdt, rf, dart, goss
#     # gbdt, 传统的梯度提升决策树
#     # rf, Random Forest (随机森林)
#     # dart, Dropouts meet Multiple Additive Regression Trees
#     # goss, Gradient-based One-Side Sampling (基于梯度的单侧采样)
#     'objective': 'binary', 
#     # options=regression, regression_l1, 
#     # huber, fair, poisson, quantile, quantile_l2, 
#     # binary, multiclass, multiclassova, xentropy, xentlambda, lambdarank
#     'metric': {'binary_logloss', 'auc'},  
#     # default={l2 for regression}, 
#     # {binary_logloss for binary classification}, {ndcg for lambdarank}, type=multi-enum, 
#     # options=l1, l2, ndcg, auc, binary_logloss, binary_error
#     'num_leaves': 40,       # default=31
#     'max_depth': -1,        # <0代表无限制  限制树模型的最大深度. 这可以在 #data 小的情况下防止过拟合. 树仍然可以通过 leaf-wise 生长.
#     'min_data_in_leaf': 450,   # default=20, 一个叶子上数据的最小数量. 可以用来处理过拟合
#     'learning_rate': 0.1,   #default=0.1
#     'feature_fraction': 0.9,  #default=1.0
#     #如果 feature_fraction 小于 1.0, LightGBM 将会在每次迭代中随机选择部分特征. 例如, 如果设置为 0.8, 
#     #将会在每棵树训练之前选择 80% 的特征
#     #可以用来加速训练  可以用来处理过拟合
#     'bagging_fraction': 0.95,  #default=1.0   0.0 < bagging_fraction < 1.0
#     #类似于 feature_fraction, 但是它将在不进行重采样的情况下随机选择部分数据
#     #可以用来加速训练  可以用来处理过拟合
#     #Note: 为了启用 bagging, bagging_freq 应该设置为非零值
#     'bagging_freq': 5,  #default=0
#     #bagging 的频率, 0 意味着禁用 bagging. k 意味着每 k 次迭代执行bagging
#     #Note: 为了启用 bagging, bagging_fraction 设置适当
#     'lambda_l1': 1,  #default=0
#     'lambda_l2': 0.001,  # 越小l2正则程度越高  default=0
#     'min_gain_to_split': 0.2,  #default=0
#     'verbose': 5,  #default=1     <0 = 致命的, =0 = 错误 (警告), >0 = 信息
#     'is_unbalance': True   #default=false 用于 binary 分类  如果培训数据不平衡 设置为 true
# }

params = {
    'task': 'train',
    'boosting_type': 'rf',
    'objective': 'binary',
    'metric': {'l2', 'auc'},
    'num_leaves': 60,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [28]:
########## 线下测试  lightGBM ############
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=300,
                valid_sets=lgb_eval,
                early_stopping_rounds=5)

[1]	valid_0's auc: 0.949327	valid_0's l2: 0.0241226
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's auc: 0.955945	valid_0's l2: 0.023827
[3]	valid_0's auc: 0.955653	valid_0's l2: 0.0238123
[4]	valid_0's auc: 0.955598	valid_0's l2: 0.0238499
[5]	valid_0's auc: 0.955777	valid_0's l2: 0.0238683
[6]	valid_0's auc: 0.955834	valid_0's l2: 0.0238173
[7]	valid_0's auc: 0.955758	valid_0's l2: 0.0237887
Early stopping, best iteration is:
[2]	valid_0's auc: 0.955945	valid_0's l2: 0.023827


In [29]:
########## 线下测试  lightGBM ############
y_test_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [30]:
########## 线下测试  lightGBM ############
print(score(y_test, y_test_pred))

0.3766086298258895


In [8]:
########## 线下测试  xgBoost ############
xgb_val = xgb.DMatrix(X_val, label=y_val)
xgb_train = xgb.DMatrix(X_train, label=y_train)
xgb_test = xgb.DMatrix(X_test)
params = {
    'booster': 'gbtree',
    # 'objective': 'multi:softmax',  # 多分类的问题、
    # 'objective': 'multi:softprob',   # 多分类概率
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    # 'num_class': 9,  # 类别数，与 multisoftmax 并用
    'gamma': 0.1,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
    'max_depth': 8,  # 构建树的深度，越大越容易过拟合
    'alpha': 0,   # L1正则化系数
    'lambda': 2,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
    'subsample': 0.7,  # 随机采样训练样本
    'colsample_bytree': 0.5,  # 生成树时进行的列采样
    'min_child_weight': 3,
    # 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
    # ，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
    # 这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。
    'silent': 0,  # 设置成1则没有运行信息输出，最好是设置为0.
    'eta': 0.03,  # 如同学习率
    'seed': 1000,
    #'nthread': -1,  # cpu 线程数
    'missing': 1,
    'scale_pos_weight': (np.sum(y==0)/np.sum(y==1))  # 用来处理正负样本不均衡的问题,通常取：sum(negative cases) / sum(positive cases)
    # 'eval_metric': 'auc'
}

plst = list(params.items())
num_rounds = 5000  # 迭代次数
watchlist = [(xgb_train, 'train'), (xgb_val, 'val')]

In [9]:
########## 线下测试  xgBoost ############
model = xgb.train(plst, xgb_train, num_rounds, watchlist, early_stopping_rounds=200)

[0]	train-logloss:0.672226	val-logloss:0.672029
Multiple eval metrics have been passed: 'val-logloss' will be used for early stopping.

Will train until val-logloss hasn't improved in 200 rounds.
[1]	train-logloss:0.651417	val-logloss:0.650772
[2]	train-logloss:0.632801	val-logloss:0.631661
[3]	train-logloss:0.614541	val-logloss:0.613151
[4]	train-logloss:0.596998	val-logloss:0.5967
[5]	train-logloss:0.580049	val-logloss:0.583807
[6]	train-logloss:0.564249	val-logloss:0.568183
[7]	train-logloss:0.548956	val-logloss:0.552945
[8]	train-logloss:0.534711	val-logloss:0.539554
[9]	train-logloss:0.521489	val-logloss:0.526078
[10]	train-logloss:0.508886	val-logloss:0.513128
[11]	train-logloss:0.496688	val-logloss:0.501194
[12]	train-logloss:0.484542	val-logloss:0.488593
[13]	train-logloss:0.473105	val-logloss:0.476938
[14]	train-logloss:0.462537	val-logloss:0.468961
[15]	train-logloss:0.452586	val-logloss:0.45884
[16]	train-logloss:0.442421	val-logloss:0.44841
[17]	train-logloss:0.432851	val-l

[164]	train-logloss:0.147577	val-logloss:0.154735
[165]	train-logloss:0.147243	val-logloss:0.154428
[166]	train-logloss:0.147047	val-logloss:0.154195
[167]	train-logloss:0.146837	val-logloss:0.153871
[168]	train-logloss:0.146611	val-logloss:0.153689
[169]	train-logloss:0.146596	val-logloss:0.153776
[170]	train-logloss:0.146487	val-logloss:0.153523
[171]	train-logloss:0.14616	val-logloss:0.153185
[172]	train-logloss:0.145757	val-logloss:0.152937
[173]	train-logloss:0.145547	val-logloss:0.152764
[174]	train-logloss:0.145557	val-logloss:0.152846
[175]	train-logloss:0.145291	val-logloss:0.152689
[176]	train-logloss:0.145006	val-logloss:0.152346
[177]	train-logloss:0.144838	val-logloss:0.152271
[178]	train-logloss:0.144533	val-logloss:0.151989
[179]	train-logloss:0.144322	val-logloss:0.151746
[180]	train-logloss:0.144282	val-logloss:0.151823
[181]	train-logloss:0.144261	val-logloss:0.151892
[182]	train-logloss:0.144128	val-logloss:0.151876
[183]	train-logloss:0.14392	val-logloss:0.151684
[1

[329]	train-logloss:0.121745	val-logloss:0.133221
[330]	train-logloss:0.121799	val-logloss:0.133276
[331]	train-logloss:0.121668	val-logloss:0.13314
[332]	train-logloss:0.121549	val-logloss:0.133084
[333]	train-logloss:0.121408	val-logloss:0.132988
[334]	train-logloss:0.121267	val-logloss:0.132834
[335]	train-logloss:0.121345	val-logloss:0.13291
[336]	train-logloss:0.121209	val-logloss:0.132811
[337]	train-logloss:0.121252	val-logloss:0.132852
[338]	train-logloss:0.120988	val-logloss:0.132582
[339]	train-logloss:0.120812	val-logloss:0.132412
[340]	train-logloss:0.120669	val-logloss:0.132278
[341]	train-logloss:0.120531	val-logloss:0.132285
[342]	train-logloss:0.120298	val-logloss:0.132048
[343]	train-logloss:0.120299	val-logloss:0.132079
[344]	train-logloss:0.120186	val-logloss:0.131998
[345]	train-logloss:0.120055	val-logloss:0.13188
[346]	train-logloss:0.11998	val-logloss:0.131541
[347]	train-logloss:0.120004	val-logloss:0.131562
[348]	train-logloss:0.119854	val-logloss:0.131362
[349

[494]	train-logloss:0.105569	val-logloss:0.118496
[495]	train-logloss:0.105542	val-logloss:0.118506
[496]	train-logloss:0.105471	val-logloss:0.11846
[497]	train-logloss:0.1055	val-logloss:0.118492
[498]	train-logloss:0.105306	val-logloss:0.118417
[499]	train-logloss:0.105202	val-logloss:0.118118
[500]	train-logloss:0.105219	val-logloss:0.118133
[501]	train-logloss:0.105229	val-logloss:0.118145
[502]	train-logloss:0.105152	val-logloss:0.118082
[503]	train-logloss:0.105174	val-logloss:0.1181
[504]	train-logloss:0.10519	val-logloss:0.118113
[505]	train-logloss:0.105019	val-logloss:0.117894
[506]	train-logloss:0.10506	val-logloss:0.117933
[507]	train-logloss:0.104883	val-logloss:0.11779
[508]	train-logloss:0.104725	val-logloss:0.117606
[509]	train-logloss:0.104499	val-logloss:0.117388
[510]	train-logloss:0.104537	val-logloss:0.117424
[511]	train-logloss:0.104441	val-logloss:0.11736
[512]	train-logloss:0.104471	val-logloss:0.117405
[513]	train-logloss:0.104379	val-logloss:0.117363
[514]	tra

[659]	train-logloss:0.093541	val-logloss:0.106465
[660]	train-logloss:0.093428	val-logloss:0.106353
[661]	train-logloss:0.093362	val-logloss:0.105934
[662]	train-logloss:0.09333	val-logloss:0.105937
[663]	train-logloss:0.093183	val-logloss:0.105819
[664]	train-logloss:0.092923	val-logloss:0.105401
[665]	train-logloss:0.092941	val-logloss:0.105419
[666]	train-logloss:0.092808	val-logloss:0.105319
[667]	train-logloss:0.092677	val-logloss:0.105188
[668]	train-logloss:0.092725	val-logloss:0.105233
[669]	train-logloss:0.092726	val-logloss:0.105235
[670]	train-logloss:0.092746	val-logloss:0.105255
[671]	train-logloss:0.09271	val-logloss:0.105235
[672]	train-logloss:0.092736	val-logloss:0.105259
[673]	train-logloss:0.092763	val-logloss:0.105284
[674]	train-logloss:0.09262	val-logloss:0.105046
[675]	train-logloss:0.092569	val-logloss:0.105016
[676]	train-logloss:0.092524	val-logloss:0.105004
[677]	train-logloss:0.092449	val-logloss:0.104937
[678]	train-logloss:0.092499	val-logloss:0.104987
[67

...

[1313]	train-logloss:0.06434	val-logloss:0.080543
[1314]	train-logloss:0.064269	val-logloss:0.080496
[1315]	train-logloss:0.064267	val-logloss:0.080494
[1316]	train-logloss:0.064197	val-logloss:0.080371
[1317]	train-logloss:0.064182	val-logloss:0.080365
[1318]	train-logloss:0.064132	val-logloss:0.08035
[1319]	train-logloss:0.064138	val-logloss:0.080355
[1320]	train-logloss:0.064081	val-logloss:0.080311
[1321]	train-logloss:0.064097	val-logloss:0.080326
[1322]	train-logloss:0.064099	val-logloss:0.080329
[1323]	train-logloss:0.064009	val-logloss:0.080262
[1324]	train-logloss:0.063913	val-logloss:0.080198
[1325]	train-logloss:0.063805	val-logloss:0.080108
[1326]	train-logloss:0.063818	val-logloss:0.08012
[1327]	train-logloss:0.06381	val-logloss:0.080123
[1328]	train-logloss:0.063814	val-logloss:0.080133
[1329]	train-logloss:0.063831	val-logloss:0.080148
[1330]	train-logloss:0.063756	val-logloss:0.080122
[1331]	train-logloss:0.06377	val-logloss:0.080135
[1332]	train-logloss:0.063739	val-lo

[1475]	train-logloss:0.059013	val-logloss:0.076368
[1476]	train-logloss:0.059025	val-logloss:0.076379
[1477]	train-logloss:0.059037	val-logloss:0.076389
[1478]	train-logloss:0.059006	val-logloss:0.076357
[1479]	train-logloss:0.058925	val-logloss:0.076283
[1480]	train-logloss:0.058918	val-logloss:0.076303
[1481]	train-logloss:0.058815	val-logloss:0.076235
[1482]	train-logloss:0.058816	val-logloss:0.076236
[1483]	train-logloss:0.058799	val-logloss:0.076234
[1484]	train-logloss:0.058756	val-logloss:0.076208
[1485]	train-logloss:0.058687	val-logloss:0.076169
[1486]	train-logloss:0.058665	val-logloss:0.076156
[1487]	train-logloss:0.058598	val-logloss:0.07611
[1488]	train-logloss:0.058585	val-logloss:0.076097
[1489]	train-logloss:0.058529	val-logloss:0.076035
[1490]	train-logloss:0.058441	val-logloss:0.075974
[1491]	train-logloss:0.058452	val-logloss:0.075985
[1492]	train-logloss:0.058374	val-logloss:0.075905
[1493]	train-logloss:0.058351	val-logloss:0.075889
[1494]	train-logloss:0.058315	va

[1637]	train-logloss:0.053386	val-logloss:0.072507
[1638]	train-logloss:0.05333	val-logloss:0.072466
[1639]	train-logloss:0.053339	val-logloss:0.072474
[1640]	train-logloss:0.053285	val-logloss:0.072443
[1641]	train-logloss:0.053195	val-logloss:0.072379
[1642]	train-logloss:0.053167	val-logloss:0.072363
[1643]	train-logloss:0.053104	val-logloss:0.072268
[1644]	train-logloss:0.053122	val-logloss:0.072283
[1645]	train-logloss:0.053131	val-logloss:0.072291
[1646]	train-logloss:0.053078	val-logloss:0.072272
[1647]	train-logloss:0.05303	val-logloss:0.072247
[1648]	train-logloss:0.052978	val-logloss:0.072216
[1649]	train-logloss:0.052952	val-logloss:0.072205
[1650]	train-logloss:0.0529	val-logloss:0.072143
[1651]	train-logloss:0.052905	val-logloss:0.072147
[1652]	train-logloss:0.052861	val-logloss:0.072105
[1653]	train-logloss:0.052841	val-logloss:0.072039
[1654]	train-logloss:0.052777	val-logloss:0.07201
[1655]	train-logloss:0.052762	val-logloss:0.071981
[1656]	train-logloss:0.052677	val-lo

[1799]	train-logloss:0.048626	val-logloss:0.069364
[1800]	train-logloss:0.048562	val-logloss:0.069346
[1801]	train-logloss:0.048547	val-logloss:0.06936
[1802]	train-logloss:0.048524	val-logloss:0.069332
[1803]	train-logloss:0.048512	val-logloss:0.06933
[1804]	train-logloss:0.048512	val-logloss:0.06933
[1805]	train-logloss:0.048509	val-logloss:0.069327
[1806]	train-logloss:0.048514	val-logloss:0.069332
[1807]	train-logloss:0.048523	val-logloss:0.06934
[1808]	train-logloss:0.048497	val-logloss:0.069321
[1809]	train-logloss:0.048441	val-logloss:0.069275
[1810]	train-logloss:0.048353	val-logloss:0.069205
[1811]	train-logloss:0.048323	val-logloss:0.069193
[1812]	train-logloss:0.04834	val-logloss:0.069208
[1813]	train-logloss:0.048292	val-logloss:0.069172
[1814]	train-logloss:0.048297	val-logloss:0.069177
[1815]	train-logloss:0.048312	val-logloss:0.06919
[1816]	train-logloss:0.048311	val-logloss:0.069196
[1817]	train-logloss:0.048266	val-logloss:0.069172
[1818]	train-logloss:0.048272	val-log

[1961]	train-logloss:0.044755	val-logloss:0.066748
[1962]	train-logloss:0.044766	val-logloss:0.066758
[1963]	train-logloss:0.044728	val-logloss:0.066721
[1964]	train-logloss:0.04474	val-logloss:0.066731
[1965]	train-logloss:0.044684	val-logloss:0.066692
[1966]	train-logloss:0.044641	val-logloss:0.066642
[1967]	train-logloss:0.044596	val-logloss:0.066622
[1968]	train-logloss:0.044602	val-logloss:0.066627
[1969]	train-logloss:0.044561	val-logloss:0.066597
[1970]	train-logloss:0.04456	val-logloss:0.066596
[1971]	train-logloss:0.044568	val-logloss:0.066603
[1972]	train-logloss:0.04458	val-logloss:0.066613
[1973]	train-logloss:0.044593	val-logloss:0.066625
[1974]	train-logloss:0.044577	val-logloss:0.066602
[1975]	train-logloss:0.044586	val-logloss:0.066611
[1976]	train-logloss:0.044589	val-logloss:0.066618
[1977]	train-logloss:0.044504	val-logloss:0.066559
[1978]	train-logloss:0.044491	val-logloss:0.066559
[1979]	train-logloss:0.044446	val-logloss:0.066526
[1980]	train-logloss:0.044372	val-

In [11]:
########## 线下测试  xgBoost ############
y_preds = model.predict(xgb_test)
print(score(y_test, y_preds))

0.3797880393641181
